# Yellow Taxi Data ingest verification - Part 5

In this notebook, we verify that the data has been loaded consistently by comparing the row counts for the data in Vast S3 and Vast DB.

<div class="alert alert-block alert-info">
💡 <b>NOTE</b>
<br/>
<ul>
<li>The <b>hive3x</b> and <b>trino</b> projects need to be running for this notebook.</li>
<li>You can run projects with `cd the-project && docker compose up -d --wait`.
</div>


In [1]:
import os
VASTDB_NYT_BUCKET = os.getenv('VASTDB_NYT_BUCKET')
VASTDB_NYT_SCHEMA = os.getenv('VASTDB_NYT_SCHEMA')
VASTDB_NYT_TABLE = os.getenv('VASTDB_NYT_TABLE')

print(VASTDB_NYT_BUCKET, VASTDB_NYT_SCHEMA, VASTDB_NYT_TABLE)

S3A_BUCKET = os.getenv('S3A_BUCKET')
S3_HIVE_TAXI_URI = os.getenv("S3A_HIVE_TAXI_URI")

print(S3_HIVE_TAXI_URI)

csnowdb nyt taxi
s3a://csnow-bucket/nyt/


## VastDB Row Count

In [2]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

%load_ext sql
%sql trino://admin@${DOCKER_HOST_OR_IP}:8443/vast  --connection_arguments '{"http_scheme":"https", "verify": false}'

Connecting to 'trino://admin@se-var-vastdb-ingest:8443/vast'

In [3]:
%%sql

select 
    format('%,d', COUNT(*)) as row_count 
from
    vast."{{VASTDB_NYT_BUCKET}}|{{VASTDB_NYT_SCHEMA}}".{{VASTDB_NYT_TABLE}}

Running query in 'trino://admin@se-var-vastdb-ingest:8443/vast'

row_count
"1,763,456,499"


## Hive Row Count

In [4]:
%sql trino://admin@${DOCKER_HOST_OR_IP}:8443/hive  --connection_arguments '{"http_scheme":"https", "verify": false}'

Connecting and switching to connection 'trino://admin@se-var-vastdb-ingest:8443/hive'

In [5]:
%%sql
    
SELECT 
    format('%,d', COUNT(*)) as row_count 
FROM 
    hive.nyt.yellow_tripdata

Running query in 'trino://admin@se-var-vastdb-ingest:8443/hive'

row_count
"1,763,456,499"
